In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
import numpy as np
import pandas as pd
import os
from datetime import datetime
import time

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, auc
import plotly.figure_factory as ff
from sklearn.preprocessing import OneHotEncoder
from data_set_params import DataSetParams
from scipy.io import wavfile
params = DataSetParams()

In [3]:
#!pip install tensorflow

In [4]:
#! unzip ~/richfield_birds_split.zip -d ~/

In [5]:
#import matplotlib.pyplot as plt

In [6]:
#data_dir = pathlib.Path('/root/tensorflow_datasets/downloads/extracted/TAR_GZ.opihi.cs.uvic.ca_sound_music_speechbya81rFcWfLSW6ey5cynqyeq2qiePcL-7asMoNO6IQ0.tar.gz/music_speech')
data_dir   = 'richfield_birds_split'#'dublin_dl_birds_split'#
categories = np.array(tf.io.gfile.listdir(data_dir))
categories = [category for category in categories if 'wav' not in category]
categories

['Common Buzzard',
 'Common Kestrel',
 'Common Snipe',
 'Eurasian Curlew',
 'European Herring Gull',
 'European Robin',
 'Meadow Pipit',
 'Mute Swan',
 'Northern Lapwing',
 'Rook',
 'Tundra Swan',
 'Tundra Swan (Bewicks)']

In [7]:
def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)

    # Note: You'll use indexing here instead of tuple unpacking to enable this 
    # to work in a TensorFlow graph.
    return parts[-2]

In [8]:
filenames = tf.io.gfile.glob(str(data_dir) + '/*/*')
#filenames = tf.io.gfile.glob('birds/*/*')
filenames = [filename for filename in filenames if 'wav' in filename]
filenames = tf.random.shuffle(filenames)

In [9]:
AUTOTUNE   = tf.data.experimental.AUTOTUNE
batch_size = 32
EPOCHS     = 1#50

In [10]:
num_classes = len(categories)

In [11]:
from tf_helpers import *

In [12]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score, top_k_accuracy_score
from sklearn.preprocessing import OneHotEncoder
from math import prod

In [13]:
def save_results(model, x_test, y_true, name, filename_run):
        model.save('models/'+filename_run+'.h5')
    
        pred_lists = model.predict(x_test)
        y_pred     = np.argmax(pred_lists, axis=-1)
        pred_df    = pd.DataFrame(pred_lists, columns = categories)
        
        softmax_prediction_df = pred_df.apply(lambda x: np.exp(x - np.max(x))/np.exp(x - np.max(x)).sum(), axis=1)
        softmax_prediction_df.to_csv('results/'+filename_run+'softmax_prediction_df.csv')
        
        cm = confusion_matrix(y_true, y_pred)
        fig = plotly_cm(cm, categories)
        fig.write_html('results/'+filename_run+'confusion_matrix.html')
        
        num_trainable    = sum([prod(w.shape) for w in model.trainable_weights])
        num_nontrainable = sum([prod(w.shape) for w in model.non_trainable_weights])
        
        onehot_data = OneHotEncoder(sparse=False)
        onehot_data = onehot_data.fit_transform(np.array(y_true).reshape(len(y_true),1))
        roc_auc = [0]*num_classes
        
        for i in range(num_classes):
            roc_auc[i] = roc_auc_score(onehot_data[:, i], softmax_prediction_df.to_numpy()[:, i])
        
        name_df = pd.DataFrame(data={
                  'model':     name}, index=[0])
        metric_df = pd.DataFrame(data={
                  'top_1_acc': [accuracy_score(y_pred, y_true)],
                  'top_5_acc': [top_k_accuracy_score(y_true, softmax_prediction_df, k=5)],
                  'precision': [precision_score(y_pred, y_true, average = 'weighted')], 
                  'f1':        [f1_score(y_pred, y_true, average = 'weighted')]
                 })
        param_df = pd.DataFrame(data={
                  'trainable_params': [num_trainable],
                  'nontrainable_params': [num_nontrainable]
                 })
        auc_df = pd.DataFrame([roc_auc], columns = ['auc_'+categories[i].replace(' ', '') for i in range(num_classes)])
        
        metric_df =  pd.concat([name_df, metric_df],axis=1)
        metric_df.to_csv('results/'+filename_run+'metric_df.csv')
        
        param_df  =  pd.concat([name_df, param_df],axis=1)
        param_df.to_csv('results/'+filename_run+'param_df.csv')
        
        auc_df    =  pd.concat([name_df, auc_df],axis=1)
        auc_df.to_csv('results/'+filename_run+'auc_df.csv')

In [14]:
def load_vgg19(input_shape):
    vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    vgg_model.trainable = False ## Not trainable weights
    #vgg_model.summary()
    x = vgg_model.output
    x = Flatten()(x) # Flatten dimensions to for use in FC layers
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x) # Dropout layer to reduce overfitting
    x = Dense(64, activation='relu')(x)
    x = Dense(num_classes, activation='softmax')(x) # Softmax for multiclass
    transfer_vgg_model = Model(inputs=vgg_model.input, outputs=x)
    transfer_vgg_model.compile(
        optimizer = optimizers.Adam(learning_rate=0.0001),
        loss      = losses.SparseCategoricalCrossentropy(),
        metrics   = 'accuracy'
        )
    return transfer_vgg_model

In [15]:
def load_resnet50(input_shape):
    resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    resnet_model.trainable = False ## Not trainable weights
    #resnet_model.summary()
    x = resnet_model.output
    x = Flatten()(x) # Flatten dimensions to for use in FC layers
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x) # Dropout layer to reduce overfitting
    x = Dense(64, activation='relu')(x)
    x = Dense(num_classes, activation='softmax')(x) # Softmax for multiclass
    transfer_resnet_model = Model(inputs=resnet_model.input, outputs=x)
    transfer_resnet_model.compile(
        optimizer = optimizers.Adam(learning_rate=0.0001),
        loss      = losses.SparseCategoricalCrossentropy(),
        metrics   = 'accuracy'
        )
    return transfer_resnet_model

In [16]:
def main_model_run(filenames, index):
    print("Index: ", index)
    
    filenames   = tf.random.shuffle(filenames)
    all_labs    = [get_label(y).numpy().decode() for y in filenames]
    filename_df = pd.DataFrame({'name': filenames.numpy(),
                                'label': all_labs})
    
    train, test = train_test_split(filename_df, test_size=0.2, stratify=filename_df[['label']])
    train_files = tf.random.shuffle(train['name'])
    test_files  = tf.random.shuffle(test['name'])

    def concat_xy(ds):
            x_tmp  = [x for x,_ in ds]
            x_tmp  = tf.stack(x_tmp)
            xs_tmp = tf.unstack(x_tmp, axis=-1)
            xs_tmp = [tf.expand_dims(x_ind, axis=-1) for x_ind in xs_tmp]
            y      = np.array([y for _,y in ds])
            return xs_tmp, y
    
    print('Getting data')
    choices  = ['Mod']
    train_ds = preprocess_dataset(train_files, choices, categories, req_width=750, single_to_rgb = True, resize = 4)
    test_ds  = preprocess_dataset(test_files,  choices, categories, req_width=750, single_to_rgb = True, resize = 4)
    
    choices = ['AbsRe', 'AbsIm', 'Ang', 'Mod']
    train_ds_mult = preprocess_dataset(train_files, choices, categories, req_width=750, resize = 4)
    test_ds_mult  = preprocess_dataset(test_files,  choices, categories, req_width=750, resize = 4)
    
    X_train, y_train = concat_xy(train_ds_mult)
    X_test,  y_test  = concat_xy(test_ds_mult)
    print("Done")
    
    num_channels = len(X_train)
    concat_shape = X_train[0].shape[1:]
    
    for spec, _ in train_ds.take(1):
        input_shape = spec.shape
        
    train_ds = train_ds.batch(batch_size)
    test_ds  = test_ds.batch(batch_size)
    train_ds = train_ds.cache().prefetch(AUTOTUNE)
    test_ds  = test_ds.cache().prefetch(AUTOTUNE)
    
    filename_idx = datetime.now().strftime("%Y%m%d-%H%M%S").replace('-', '_')+'_'+data_dir+'_'+str(index)
    
    if not os.path.isdir('results'):
        os.mkdir('results')
    
    np.save('results/'+filename_idx+'_filenames.npy', filenames.numpy())
    
    #this will save the model performing best on val accuracy
    def best_model_cp():
        checkpoint = tf.keras.callbacks.ModelCheckpoint(
            "best_model",
            monitor = "val_accuracy",
            mode    = "max",
            save_best_only = True,
            save_weights_only = True)
        return checkpoint
    
    ## Load and run models
    
    #VGG19
    print("VGG19")
    model    = load_vgg19(input_shape)
    model_name   = 'vgg19'
    filename_run = filename_idx+'_'+model_name
    
    history = model.fit(train_ds,
                        validation_data = test_ds,
                        callbacks       = [best_model_cp()],
                        epochs          = EPOCHS)
    
    pd.DataFrame(history.history).to_csv('results/'+filename_run+'_model_history.csv')
    
    model.load_weights("best_model")
    
    save_results(model, test_ds, y_test, model_name, filename_run)

    #ResNet50
    print("ResNet50")
    model = load_resnet50(input_shape)
    model_name   = 'resnet50'
    filename_run = filename_idx+'_'+model_name
    
    history = model.fit(train_ds,
                        validation_data = test_ds,
                        callbacks       = [best_model_cp()],
                        epochs          = EPOCHS)

    pd.DataFrame(history.history).to_csv('results/'+filename_run+'_model_history.csv')
    
    model.load_weights("best_model")

    save_results(model, test_ds, y_test, model_name, filename_run)

    #small_cnn
    print("Small CNN")
    model  = main_cnn(input_shape, num_classes)
    model_name   = 'smallcnn'
    filename_run = filename_idx+'_'+model_name
    
    history = model.fit(train_ds,
                        validation_data = test_ds,
                        callbacks       = [best_model_cp()],
                        epochs          = EPOCHS)

    pd.DataFrame(history.history).to_csv('results/'+filename_run+'_model_history.csv')
    
    model.load_weights("best_model")

    save_results(model, test_ds, y_test, model_name, filename_run)
    
    #concat
    print("Concat")
    model      = concat_model(concat_shape, num_channels, num_classes)
    model_name   = 'concat'
    filename_run = filename_idx+'_'+model_name
    
    history = model.fit(X_train, y_train,
                        validation_data = (X_test, y_test),
                        callbacks       = [best_model_cp()],
                        epochs          = EPOCHS,
                        batch_size      = batch_size)

    pd.DataFrame(history.history).to_csv('results/'+filename_run+'_model_history.csv')
    
    model.load_weights("best_model")
    
    save_results(model, X_test, y_test, model_name, filename_run)

    #concat2
    print("Concat2")
    model     = concat_model2(concat_shape, num_channels, num_classes)
    model_name   = 'concat2'
    filename_run = filename_idx+'_'+model_name
    
    history = model.fit(X_train, y_train,
                        validation_data = (X_test, y_test),
                        callbacks       = [best_model_cp()],
                        epochs          = EPOCHS,
                        batch_size      = batch_size)

    pd.DataFrame(history.history).to_csv('results/'+filename_run+'_model_history.csv')
    
    model.load_weights("best_model")
    
    save_results(model, X_test, y_test, model_name, filename_run)

## Evaluate

In [17]:
tic = time.time()

In [18]:
for i in range(6):
    main_model_run(filenames, i)
    print("Time so far:", time.time()-tic)

Index:  0
Getting data
Done
VGG19
ResNet50
71/71 [==============================] - 105s 1s/step - loss: 2.0892 - accuracy: 0.2953 - val_loss: 1.7501 - val_accuracy: 0.4102


C:\Users\Anthony\Anaconda3\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning:

Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.



Small CNN
71/71 [==============================] - 14s 198ms/step - loss: 2.6032 - accuracy: 0.1536 - val_loss: 2.3891 - val_accuracy: 0.2570
Concat
71/71 [==============================] - 56s 784ms/step - loss: 2.3790 - accuracy: 0.2535 - val_loss: 2.0783 - val_accuracy: 0.4243
Concat2
71/71 [==============================] - 63s 872ms/step - loss: 2.5813 - accuracy: 0.1417 - val_loss: 2.4126 - val_accuracy: 0.2606
Time so far: 413.64055919647217
Index:  1
Getting data


KeyboardInterrupt: 

In [ ]:
toc = time.time()

In [ ]:
print(toc-tic)

In [ ]:
#print(res_df_t.to_latex(bold_rows = True))

In [18]:
index=1
print("Index: ", index)

filenames   = tf.random.shuffle(filenames)
all_labs    = [get_label(y).numpy().decode() for y in filenames]
filename_df = pd.DataFrame({'name': filenames.numpy(),
                            'label': all_labs})

train, test = train_test_split(filename_df, test_size=0.2, stratify=filename_df[['label']])
train_files = tf.random.shuffle(train['name'])
test_files  = tf.random.shuffle(test['name'])

def concat_xy(ds):
        x_tmp  = [x for x,_ in ds]
        x_tmp  = tf.stack(x_tmp)
        xs_tmp = tf.unstack(x_tmp, axis=-1)
        xs_tmp = [tf.expand_dims(x_ind, axis=-1) for x_ind in xs_tmp]
        y      = np.array([y for _,y in ds])
        return xs_tmp, y

print('Getting data')
choices  = ['Mod']
train_ds = preprocess_dataset(train_files, choices, categories, req_width=750, single_to_rgb = True, resize = 4)
test_ds  = preprocess_dataset(test_files,  choices, categories, req_width=750, single_to_rgb = True, resize = 4)

#choices = ['AbsRe', 'AbsIm', 'Ang', 'Mod']
#train_ds_mult = preprocess_dataset(train_files, choices, categories, req_width=750, resize = 4)
#test_ds_mult  = preprocess_dataset(test_files,  choices, categories, req_width=750, resize = 4)

#X_train, y_train = concat_xy(train_ds_mult)
#X_test,  y_test  = concat_xy(test_ds_mult)
print("Done")

num_channels = len(X_train)
concat_shape = X_train[0].shape[1:]

for spec, _ in train_ds.take(1):
    input_shape = spec.shape

train_ds = train_ds.batch(batch_size)
test_ds  = test_ds.batch(batch_size)
train_ds = train_ds.cache().prefetch(AUTOTUNE)
test_ds  = test_ds.cache().prefetch(AUTOTUNE)


Index:  1
Getting data
Done


In [20]:
model  = main_cnn(input_shape, num_classes)
model_name   = 'smallcnn'
#filename_run = filename_idx+'_'+model_name

model.evaluate(test_ds)
#history = model.fit(train_ds,
#                    validation_data = test_ds,
#                    callbacks       = [best_model_cp()],
#                    epochs          = EPOCHS)

18/18 [==============================] - 14s 602ms/step - loss: 2.6678 - accuracy: 0.0651


[2.6678380966186523, 0.06514084339141846]